## ISABELA VICTÓRIA DE NOVAIS ROMANATO - RM: 550234

# Exercício Reg Linear e Reg Polinomial

Entrega: 24/3/24

Analisar a base de dados IMDB, entregue em aula.| OK

Utilizar como variáveis regressoras:
1. Receita (revenue) | OK
2. Orçamento (budget) | OK
3. Língua Original (original_language) | OK
4. País de produção (production_coutries) | OK
5. Data de lançamento (release_date) | OK

Tratamento:
1. vote_count > 500 | OK
2. original_language (== EN || != EN) | OK
3. revenue (tratamento para função exponencial) | OK
4. budget (tratamento para função exponencial) | OK
5. production_countries (contém EUA || não contém EUA) | OK
6. release_date (> 2000 || < 2000) | OK
7. Informações zeradas: cortar a linha toda! | OK

Variável de interesse: *média da nota do filme* | OK

Criar modelo de regressão linear e modelo de regressão logística. | OK

Utilizar 50% da base como treino, separar da base total. | OK
Restante das observações, utilizar 20 amostras parciais com 10% do total. | OK

Avaliar se score de teste é estatisticamente igual ao score de treino.

Verificar qual modelo utilizar e justificar (1 parágrafo).

Exemplo:
Se fossem 1000 observações:
500 => somente para treino
500 => gerar 20 amostras com 10% do total (20 amostras de 50 observ | OK

In [114]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import seaborn as sns
import statsmodels.api as sm
import scipy.stats .stats as stats
from sklearn.preprocessing import PolynomialFeatures

In [115]:
df_original = pd.read_csv('/content/drive/My Drive/Colab Notebooks/2TSCF - 2024/MACHINE LEARNING/TMDB_all_movies.csv', quotechar='"', sep=',')
df_original

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,budget,imdb_id,...,genres,production_companies,production_countries,spoken_languages,cast,director,director_of_photography,writers,producers,music_composer
0,2,Ariel,7.082,304,Released,1988-10-21,0.0,73,0.0,tt0094675,...,"Drama, Comedy, Romance, Crime",Villealfa Filmproductions,Finland,suomi,"Markku Rantala, Esko Nikkari, Matti Pellonpää,...",Aki Kaurismäki,Timo Salminen,Aki Kaurismäki,Aki Kaurismäki,NaN
1,3,Shadows in Paradise,7.302,344,Released,1986-10-17,0.0,74,0.0,tt0092149,...,"Drama, Comedy, Romance",Villealfa Filmproductions,Finland,"suomi, English, svenska","Esko Nikkari, Matti Pellonpää, Sirkka Silin, H...",Aki Kaurismäki,Timo Salminen,Aki Kaurismäki,Mika Kaurismäki,NaN
2,5,Four Rooms,5.800,2534,Released,1995-12-09,4257354.0,98,4000000.0,tt0113101,...,Comedy,"Miramax, A Band Apart",United States of America,English,"Amanda de Cadenet, Antonio Banderas, Madonna, ...","Alexandre Rockwell, Robert Rodriguez, Quentin ...","Guillermo Navarro, Phil Parmet, Rodrigo García...","Alexandre Rockwell, Robert Rodriguez, Quentin ...","Alexandre Rockwell, Lawrence Bender, Quentin T...",Combustible Edison
3,6,Judgment Night,6.527,316,Released,1993-10-15,12136938.0,109,21000000.0,tt0107286,...,"Action, Crime, Thriller","Largo Entertainment, JVC, Universal Pictures",United States of America,English,"Everlast, Peter Greene, Michael DeLorenzo, Den...",Stephen Hopkins,Peter Levy,"Jere Cunningham, Lewis Colick","Marilyn Vance, Gene Levy, Lloyd Segan",Alan Silvestri
4,8,Life in Loops (A Megacities RMX),7.500,27,Released,2006-01-01,0.0,80,42000.0,tt0825671,...,Documentary,inLoops,Austria,"English, हिन्दी, 日本語, Pусский, Español",NaN,Timo Novotny,Wolfgang Thaler,"Michael Glawogger, Timo Novotny","Ulrich Gehmacher, Timo Novotny",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
906542,1260330,月遊戯 8,0.000,0,Released,NaN,0.0,60,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
906543,1260331,Самые красивые корабли,0.000,0,Released,NaN,0.0,0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
906544,1260332,Errand,0.000,0,Released,NaN,0.0,0,0.0,tt14661424,...,NaN,NaN,NaN,NaN,NaN,Jared Hawkley,NaN,Jared Hawkley,NaN,NaN
906545,1260333,تنهایی,0.000,0,Released,NaN,0.0,90,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [116]:
# FILTRAR DF COM A COLUNA vote_count > 500

df = df_original[df_original['vote_count'] > 500]
df.head(2)

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,budget,imdb_id,...,genres,production_companies,production_countries,spoken_languages,cast,director,director_of_photography,writers,producers,music_composer
2,5,Four Rooms,5.800,2534,Released,1995-12-09,4257354.0,98,4000000.0,tt0113101,...,Comedy,"Miramax, A Band Apart",United States of America,English,"Amanda de Cadenet, Antonio Banderas, Madonna, ...","Alexandre Rockwell, Robert Rodriguez, Quentin ...","Guillermo Navarro, Phil Parmet, Rodrigo García...","Alexandre Rockwell, Robert Rodriguez, Quentin ...","Alexandre Rockwell, Lawrence Bender, Quentin T...",Combustible Edison
6,11,Star Wars,8.204,19763,Released,1977-05-25,775398007.0,121,11000000.0,tt0076759,...,"Adventure, Action, Science Fiction","Lucasfilm Ltd., 20th Century Fox",United States of America,English,"Morgan Upton, James Earl Jones, Warwick Diamon...",George Lucas,Gilbert Taylor,George Lucas,"Gary Kurtz, Rick McCallum, George Lucas",John Williams


In [117]:
# ELIMINAR AS LINHAS COM VALORES AUSENTES

df = df.dropna(how='any', axis=0)

In [118]:
#  PARA AS COLUNAS: 'revenue' E 'budget' REALIZAR TRATAMENTO PARA FUNÇÃO EXPONENCIAL

df['log_revenue'] = np.log(df['revenue'])
df['log_budget'] = np.log(df['budget'])

/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [119]:
# TRANSFORMAR VARIÁVEIS CATEGÓRICAS

# original_language (== EN || != EN)
df['original_language'] = df.apply (lambda row: 1 if row ['original_language'] == 'en' else 0, axis=1)

# production_countries (contém EUA || não contém EUA)
df['production_countries'] = df.apply (lambda row: 1 if 'United States of America' in row ['production_countries'] else 0, axis=1)

# release_date (> 2000 || < 2000)
df['release_date'] = df.apply (lambda row: 1 if row ['release_date'] > '2000' else 0, axis=1)


In [120]:
#saber quantas linhas
df.count()

id                         4380
title                      4380
vote_average               4380
vote_count                 4380
status                     4380
release_date               4380
revenue                    4380
runtime                    4380
budget                     4380
imdb_id                    4380
original_language          4380
original_title             4380
overview                   4380
popularity                 4380
tagline                    4380
genres                     4380
production_companies       4380
production_countries       4380
spoken_languages           4380
cast                       4380
director                   4380
director_of_photography    4380
writers                    4380
producers                  4380
music_composer             4380
log_revenue                4380
log_budget                 4380
dtype: int64

In [121]:
# Dividir minha base total em teste e treino

#saber quantas linhas
#df.count() #4380

# dividir minha base total em base de treino - usar metade
df_train = df.sample(2190)

#retirar da base principal os meus treinos
df = df.drop(df_train.index)

#crio uma lista com 20 amostras e 219 observações cada a apartir das 2190 obs que sobraram
# após a retirada das obs de treino. HAVERÁ REPETIÇÃO
samples = []
for i in range(20):
  samples.append(df.sample(219))

  print(samples)

A saída de streaming foi truncada nas últimas 5000 linhas.
21827                   Mike Eley   
126                   Dean Cundey   
375212              Tat Radcliffe   
145407              Shane Hurlbut   

                                                  writers  \
837     Daniel F. Galouye, Josef Rusnak, Ravel Centeno...   
4480                                           Ian McEwan   
2959                         Ross LaManna, Jeff Nathanson   
5801    William Douglas-Home, Jenny Bicks, Elizabeth C...   
264889  Gina Prince-Bythewood, Lauren Oliver, Maria Ma...   
...                                                   ...   
131                                          Alex Garland   
21827                    Christianna Brand, Emma Thompson   
126                             Bob Gale, Robert Zemeckis   
375212                            Lena Waithe, James Frey   
145407                                         Brad Desch   

                                                producers  

In [122]:
df.columns

Index(['id', 'title', 'vote_average', 'vote_count', 'status', 'release_date',
       'revenue', 'runtime', 'budget', 'imdb_id', 'original_language',
       'original_title', 'overview', 'popularity', 'tagline', 'genres',
       'production_companies', 'production_countries', 'spoken_languages',
       'cast', 'director', 'director_of_photography', 'writers', 'producers',
       'music_composer', 'log_revenue', 'log_budget'],
      dtype='object')

In [123]:
# VARIÁVEIS REGRESSORAS
X_train = df_train[['revenue', 'budget', 'original_language', 'production_countries', 'release_date']].to_numpy()
# VARIÁVEL DE INTERESSE
Y_train = df_train[['vote_average']].to_numpy()

In [124]:
# MODELO DE REGRESSÃO LINEAR

model_linear = LinearRegression()
model_linear.fit(X_train, Y_train)
score_linear = model_linear.score(X_train, Y_train)
print(score_linear)

0.17920285019212467


In [125]:
# MODELO DE REGRESSÃO POLINOMIAL

poly_features = PolynomialFeatures(degree=2)
X_poly_train = poly_features.fit_transform(X_train)
model_poly = LinearRegression()
model_poly.fit(X_poly_train, Y_train)
score_polynomial = model_poly.score(X_poly_train, Y_train)
print(score_polynomial)

0.11768974313553027


In [126]:
# TREINAR MODELO

test_score_linear = []
test_score_polynomial = []
for sample in samples:
  # CRIANDO VARIÁVEIS 'X', 'Y' PARA REGRESSÃO LINEAR
  X_test = sample [['revenue', 'budget', 'original_language', 'production_countries', 'release_date']].to_numpy()
  Y_test = sample [['vote_average']].to_numpy()
  test_score_linear.append(model_linear.score(X_test, Y_test))

 # CRIANDO VARIÁVEL POLINOMIAL PARA O X
X_test_poly = poly_features.fit_transform(X_test)
test_score_polynomial.append(model_poly.score(X_test_poly, Y_test))

alpha = 0.1

In [129]:
# MODELAGEM DO TESTE DE HIPÓTESES
# H0: score de teste é igual ao score de treino
# a minha regressão não perde poder de explicação quando testa novas variávies
# H1: score de teste é diferente  do score de treino
# a minha regressão não explica a variável de interesse da mesma maneira que o teste
t_stat, p_valor = stats.ttest_1samp(test_score_linear, popmean=score_linear)
print("LINEAR")
print('score_linear', score_linear)
print('test_score_linear',test_score_linear)
if p_valor < alpha:
  print('Regressão Linear: Aceite H1 e rejeito H0')
else:
  print('Regressão Linear: Aceito H0 e rejeito H1')

t_stat, p_valor = stats.ttest_1samp(test_score_polynomial, popmean=score_polynomial)
print("POLINOMIAL")
print('score_polynomial', score_polynomial)
print('test_score_polynomial',test_score_polynomial)
if p_valor < alpha:
  print('Regressão Polinomial: Aceite H1 e rejeito H0')
else:
  print('Regressão Polinomial: Aceito H0 e rejeito H1')

LINEAR
score_linear 0.17920285019212467
test_score_linear [0.12895324900198446, 0.23332860251728393, 0.19603171977923184, 0.13592732300087906, 0.16922236867242113, 0.19369451138100025, 0.1635022509666607, 0.25698050124807215, 0.1021645275132902, 0.18422437728921925, 0.13669552611218738, 0.13977133532413055, 0.19794538295674158, 0.20972982073437096, 0.14198350030530826, 0.25912672592854535, 0.21321250345976972, 0.21837252938316132, 0.17013145804776053, 0.2183923040329513]
Regressão Linear: Aceito H0 e rejeito H1
POLINOMIAL
score_polynomial 0.11768974313553027
test_score_polynomial [0.10754260281119377]
Regressão Polinomial: Aceito H0 e rejeito H1


<ipython-input-129-6e909837c707>:6: DeprecationWarning: Please use `ttest_1samp` from the `scipy.stats` namespace, the `scipy.stats.stats` namespace is deprecated.
  t_stat, p_valor = stats.ttest_1samp(test_score_linear, popmean=score_linear)
<ipython-input-129-6e909837c707>:15: DeprecationWarning: Please use `ttest_1samp` from the `scipy.stats` namespace, the `scipy.stats.stats` namespace is deprecated.
  t_stat, p_valor = stats.ttest_1samp(test_score_polynomial, popmean=score_polynomial)


O resultado apresenta uma avaliação do desempenho de dois modelos de regressão, linear e polinomial, com base nos scores de treino e teste. O score de teste é comparado ao score de treino para determinar se há uma diferença estatisticamente significativa entre eles. Para a regressão linear, o teste estatístico sugere que não há diferença significativa entre os scores de teste e treino, ressaltando que para a regressão polinomial, a hipótese nula é aceita, indicando que os scores de teste e treino são estatisticamente iguais. A regressão polinomial é preferível devido à sua capacidade intrínseca de modelar relações não lineares, o que pode levar a uma melhor capacidade de generalização em novos dados não vistos.




